In [ ]:
from pyomo.environ import *

# Datos de entrada
generators = [
    {"name": "G1", "Pmax": 100, "Pmin": 30, "a": 0.01, "b": 10, "c": 200, "UR": 20, "DR": 15, "P_prev": 50},
    {"name": "G2", "Pmax": 150, "Pmin": 50, "a": 0.02, "b": 8, "c": 300, "UR": 30, "DR": 20, "P_prev": 100},
    {"name": "G3", "Pmax": 120, "Pmin": 40, "a": 0.015, "b": 9, "c": 250, "UR": 25, "DR": 18, "P_prev": 70}
]

P_demand = 250  # Demanda total en MW
P_losses = 10    # Pérdidas estimadas en MW

# Crear modelo
model = ConcreteModel()

# Índices
gen_ids = range(len(generators))

# Variables
def P_bounds(model, i):
    return (generators[i]["Pmin"], generators[i]["Pmax"])

model.P = Var(gen_ids, domain=NonNegativeReals, bounds=P_bounds)

# Función objetivo
def cost_function(model):
    return sum(generators[i]["a"] * model.P[i]**2 + generators[i]["b"] * model.P[i] + generators[i]["c"] for i in gen_ids)

model.cost = Objective(rule=cost_function, sense=minimize)

# Restricción de balance de potencia
def balance_rule(model):
    return sum(model.P[i] for i in gen_ids) == P_demand + P_losses

model.balance = Constraint(rule=balance_rule)

# Restricciones de rampa
def ramp_up_rule(model, i):
    return model.P[i] - generators[i]["P_prev"] <= generators[i]["UR"]

def ramp_down_rule(model, i):
    return generators[i]["P_prev"] - model.P[i] <= generators[i]["DR"]

model.ramp_up = Constraint(gen_ids, rule=ramp_up_rule)
model.ramp_down = Constraint(gen_ids, rule=ramp_down_rule)

# Resolver el problema
solver = SolverFactory('glpk')
solver.solve(model)

# Resultados
for i, g in enumerate(generators):
    print(f"{g['name']}: {model.P[i]():.2f} MW")

print(f"Costo total: ${model.cost():.2f}")
